# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [4]:
# Instalamos el modelo predictivo Prophet
!python -m pip install prophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [5]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [6]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-05-10'
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-USD',start_date, today).reset_index()

btc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2681,2023-05-05,"$28,851.48","$29,668.91","$28,845.51","$29,534.38","$29,534.38",17936566518
2682,2023-05-06,"$29,538.86","$29,820.13","$28,468.97","$28,904.62","$28,904.62",15913866714
2683,2023-05-07,"$28,901.62","$29,157.52","$28,441.37","$28,454.98","$28,454.98",11301355486
2684,2023-05-08,"$28,450.46","$28,663.27","$27,310.13","$27,694.27","$27,694.27",19122903752
2685,2023-05-09,"$27,695.07","$27,821.40","$27,375.60","$27,658.78","$27,658.78",14128593256


In [7]:
# Validamos que no hay datos vacíos
btc_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [8]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = btc_df[['Date', 'Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df

df = df.rename(columns={"Date": "ds", "Open": "y"}) 

In [9]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [10]:
df.tail()

,ds,y
2681,2023-05-05,"$28,851.48"
2682,2023-05-06,"$29,538.86"
2683,2023-05-07,"$28,901.62"
2684,2023-05-08,"$28,450.46"
2685,2023-05-09,"$27,695.07"


In [11]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "Bitcoin y su evolución en el tiempo"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [12]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode='multiplicative')

In [13]:
#TAREA: Entrena el modelo m con el dataframe de datos df

m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwse1gbwk/e5e39ve2.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwse1gbwk/bhar8rde.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=60160', 'data', 'file=/tmp/tmpwse1gbwk/e5e39ve2.json', 'init=/tmp/tmpwse1gbwk/bhar8rde.json', 'output', 'file=/tmp/tmpwse1gbwk/prophet_modelvpgobi90/prophet_model-20230518185433.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:54:33 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:54:35 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [14]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=365)
future.tail() 


,ds
3046,2024-05-04
3047,2024-05-05
3048,2024-05-06
3049,2024-05-07
3050,2024-05-08


In [15]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-01
1,2016-01-02
2,2016-01-03
3,2016-01-04
4,2016-01-05
...,...
3046,2024-05-04
3047,2024-05-05
3048,2024-05-06
3049,2024-05-07


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [16]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [17]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
3046,2024-05-04,"$-9,593.31","$-26,526.09","$9,415.55"
3047,2024-05-05,"$-9,561.32","$-25,651.15","$10,067.32"
3048,2024-05-06,"$-9,519.11","$-25,877.74","$8,567.12"
3049,2024-05-07,"$-9,460.99","$-25,027.54","$8,973.79"
3050,2024-05-08,"$-9,373.50","$-24,624.11","$9,498.46"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [18]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [19]:
plot_components_plotly(m, forecast)